미니 프로젝트

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)

image = widgets.Image(format='jpeg', width=224, height=224)  # this width and height doesn't necessarily have to match the camera

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

#카메라 인스턴스 생성

In [ ]:
import os

red_dir = "dataset2/red"
no_dir = "dataset2/no"
blue_dir = "dataset2/blue"

# 디렉토리가 이미 존재하는 경우 다음 함수가 오류를 발생시킬 수 있기 때문에 "try/except" 문을 사용.
try:
    os.makedirs(red_dir)
    os.makedirs(no_dir)
    os.makedirs(blue_dir)
except FileExistsError:
    print('Directories not created becasue they already exist')

In [ ]:
button_layout =  widgets.Layout(width='128px',height='64px')

red_button = widgets.Button(description='add RED',button_style='success',layout=button_layout)
red_button.style.button_color = 'red'

blue_button = widgets.Button(description='add BLUE',button_style='success',layout=button_layout)
blue_button.style.button_color = 'blue'

no_button = widgets.Button(description='add NO',button_style='success',layout=button_layout)
no_button.style.button_color = 'green'

red_count = widgets.IntText(layout=button_layout,value=len(os.listdir(red_dir)))
blue_count = widgets.IntText(layout=button_layout,value=len(os.listdir(blue_dir)))
no_count = widgets.IntText(layout=button_layout,value=len(os.listdir(no_dir)))

display(widgets.HBox([red_count, red_button]))
display(widgets.HBox([blue_count, blue_button]))
display(widgets.HBox([no_count, no_button]))
# 각 사진을 저장할 수 있는 버튼과, 사진의 수량을 보여줄 텍스트 박스를 추가함.
# 미로를 구성 할 파란색, 끝을 표현할 빨간색, 아무것도 없을 때를 판단하기 위한 no.

In [ ]:
from uuid import uuid1

def save_snapshot(directory):
    image_path = os.path.join(directory, str(uuid1()) + '.jpg')
    with open(image_path, 'wb') as f:
        f.write(image.value)

def save_blue():
    global blue_dir, blue_count
    save_snapshot(blue_dir)
    blue_count.value = len(os.listdir(blue_dir))

def save_red():
    global red_dir, red_count
    save_snapshot(red_dir)
    red_count.value = len(os.listdir(red_dir))

def save_no():
    global no_dir, no_count
    save_snapshot(no_dir)
    no_count.value = len(os.listdir(no_dir))


blue_button.on_click(lambda x: save_blue())
red_button.on_click(lambda x: save_red())
no_button.on_click(lambda x: save_no())

#각 버튼 기능에 맞는 사진 저장 함수. on_Click이벤트에 등록.
#즉, 버튼 누르면 저장됨.

In [ ]:
display(image)
display(widgets.HBox([red_count, red_button]))
display(widgets.HBox([blue_count, blue_button]))
display(widgets.HBox([no_count, no_button]))

#카메라 보여주기--> 버튼 누르면, add photo 됨.

In [ ]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as transforms

dataset = datasets.ImageFolder(
    'dataset2',
    transforms.Compose([
        transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
)

In [ ]:
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [len(dataset) - 10, 10])

In [ ]:
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=16,
    shuffle=True,
    num_workers=4
)

In [ ]:
model = models.alexnet(pretrained=True)

In [ ]:
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 3)

In [ ]:
device = torch.device('cuda')
model = model.to(device)

In [ ]:
NUM_EPOCHS = 30
BEST_MODEL_PATH = 'best_model.pth'
best_accuracy = 0.0

optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

for epoch in range(NUM_EPOCHS):

    for images, labels in iter(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = F.cross_entropy(outputs, labels)
        loss.backward()
        optimizer.step()

    test_error_count = 0.0
    for images, labels in iter(test_loader):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        test_error_count += float(torch.sum(torch.abs(labels - outputs.argmax(1))))

    test_accuracy = 1.0 - float(test_error_count) / float(len(test_dataset))
    print('%d: %f' % (epoch, test_accuracy))
    if test_accuracy > best_accuracy:
        torch.save(model.state_dict(), BEST_MODEL_PATH)
        best_accuracy = test_accuracy

In [ ]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 3)

In [ ]:
model.load_state_dict(torch.load('best_model.pth'))

In [ ]:
device = torch.device('cuda')
model = model.to(device)

In [ ]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

실행

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224,fps=10)
image = widgets.Image(format='jpeg', width=224, height=224)

blue_slider = widgets.FloatSlider(description='blue', min=0.0, max=1.0, orientation='vertical')
red_slider = widgets.FloatSlider(description='red', min=0.0, max=1.0, orientation='vertical')
no_slider = widgets.FloatSlider(description='no', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, blue_slider,red_slider,no_slider]))

In [ ]:
from jetbot import Robot
import time

robot = Robot()

In [ ]:
import torch.nn.functional as F

def move(robot, direction):
  # 로봇을 주어진 방향으로 움직이는 함수 (예: "forward", "left", "right","back")
    if direction == "forward":
    robot.set_motors(1.25,1.58)
    time.sleep(0.88)
    robot.stop()
    
    if direction == "left":
    robot.set_motors(-1.65,1.3)
    time.sleep(0.36)
    robot.stop()

    if direction == "right":
    robot.set_motors(1.65,-1.3)
    time.sleep(0.36)
    robot.stop()
    
    if direction == "back":
    robot.set_motors(-1.25,-1.58)
    time.sleep(0.88)
    robot.stop()

def update(change):
    global blocked_slider, robot
    x = change['new']
    x = preprocess(x)
    y = model(x)

    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    prob_blue = float(y.flatten()[2])
    prob_red = float(y.flatten()[1])
    prob_no = float(y.flatten()[0])

    if prob_blue < 0.6:
          return "prob_blue"
    elif prob_red < 0.6:
          return "prob_red"
    else:
          return "no"

In [ ]:
# 갈림길 부터 길 저장 하는 과정 재귀함수로 작성 예정 --> 트리 순회와 동일하게 생각해보기 
# 갈림길 확인 필요하므로 리스트 따로 만들어서, len()으로 2개 이상이면 그 자리부터 스택에 저장, 앞, 오, 왼 순서로 순회.
# 재귀함수 내에 리스트 만든뒤 append, pop 써서 스택처럼 쓰기. 
# 끝 지점 빨간색 보면 종료. 이동마다 이동 메커니즘 저장. 백트랙킹에서는 pop하면서, 옳게 간 것만 저장함, 그 뒤 빨간색을 보면 생성된 최단경로로 다시 처음으로 이동 pop 하면서 move함수 이용하기. 그냥 메커니즘과 같음.
# 필요한 리스트 종류는 3개. 1번, 최단거리 저장. 2번 백트래킹용, 3번 갈림길 존재 유무 확인용.
# 월요일에 데이터셋 생성하고, 학습까지 시키자. 일단 짱규식 ta가 준 학습 코드 보기.
# 애초에 너무 데이터셋이 많아서 그런것일지도 모르니, 20~30개 정도만 만들자. 

In [ ]:
def move(robot, direction):
  # 로봇을 주어진 방향으로 움직이는 함수 (예: "forward", "left", "right","back")
    if direction == "forward":
        robot.set_motors(1.25,1.58)
        time.sleep(0.88)
        robot.stop()
    
    if direction == "left":
        robot.set_motors(-1.65,1.3)
        time.sleep(0.36)
        robot.stop()

    if direction == "right":
        robot.set_motors(1.65,-1.3)
        time.sleep(0.36)
        robot.stop()
    
    if direction == "back":
        robot.set_motors(-1.25,-1.58)
        time.sleep(0.88)
        robot.stop()

def is_crossroads(robot): #정상적인 저장, just 저장이므로 앞으로 이동하진 않음.
    crossroad = list()
    if update({'new': camera.value}) == "prob_blue":
        crossroad.append("foward")
        move(robot, "right")
            
    else:
        move(robot, "right")
        
    if update({'new': camera.value}) == "prob_blue":
        crossroad.append("right")
        move(robot, "left")
        move(robot, "left")
    else:
        move(robot, "left")
        move(robot, "left")     
        
    if update({'new': camera.value}) == "prob_blue":
        crossroad.append("left")
        move(robot, "right")
    else:
        move(robot, "right")
    return crossroad # 갈림길에서 앞,오,왼중 어디에 길이 있는지 알려주는 스택, 마지막엔 젯봇이 앞을 보고 끝

def backtracking(trever): #암, 왼, 오가 각각 다른것이므로 상관 x(오앞, 왼앞 문제)
    global total_check ,trever
    for i in trever: #이미 거꾸로 저장 중 ,이번엔 그냥 뽑고 가기가 맞음.
        if i == "back":
            total_check.pop()
            trever.pop() #pop되는 대상은 back임.
            move(robot, "back")
        if i == "right":
            total_check.pop()
            trever.pop()
            move(robot, "right")
        if i == "left":
            total_check.pop()
            trever.pop()
            move(robot, "left")

def to_back_explore(robot): #갈림길에서 길 저장
    global total_check ,trever
    while True: #갈림길이 한 경로에 2개이상인 미로에는 실행 불가하다는 단점.
        crossroad = is_crossroads(robot)
        crossroad = list(reversed(crossroad))
        new = update({'new': camera.value}) # 그 시점에 값 저장
        if  new == "prob_red":
            camera.stop()
            camera.unobserve(update_image, names='value')
            break
        if len(crossroad) == 0:
            return backtracking(trever)
        
        now = crossroad.pop()
        if now == "forward":
            total_check.append("back") #거꾸로 저장
            trever.append("back") #거꾸로 저장
            move(robot, "forward")
        if now == "right":
            total_check.append("left")
            trever.append("left")
            move(robot, "right")
        if now == "left":
            total_check.append("right")
            trever.append("right")
            move(robot, "left")
        
def total_move(total_check): #스택처럼 사용하기 위해 리버스함. for 문 쓰면 앞부터 접근.
    total_check = list(reversed(total_check))
    for i in total_check:
        
        if i == "back":
            move(robot, "back") 
        if i == "right":
            move(robot, "right")
        if i == "left":
            move(robot, "left")
            
def explore(robot): #미로 순회 함수
    global total_check ,trever
    while True:
        camera.observe(update, names='value')
        camera.start()
        #빨간색 도착 지점에 도달한 경우, 미로 탈출 성공
        new = update({'new': camera.value}) # 그 시점에 값 저장
        
        if  new == "prob_red":
            camera.stop()
            camera.unobserve(update_image, names='value')
            break
            
        else:
            crossroad = is_crossroads(robot)
            #큐 로 사용하기 위함. pop, append
            crossroad = list(reversed(crossroad)) #최소 0개~최대 3개
            #3가지의 경우의 수 길이 1개 혹은, 0개 혹은, 1개 이상. 0개는 이론상 나올 수 없음.(출구없는 미로) 그러므로 제외.
            if len(crossroad) == 1: #1개는 무조건 이동
                now = crossroad.pop()
                if now == "forward":
                    total_check.append("back")
                    move(robot, "forward")
                    
                if now == "right":
                    total_check.append("left")
                    move(robot, "right")
                    move(robot, "forward")
                
                if now == "left":
                    total_check.append("right")
                    move(robot, "left")
                    move(robot, "forward")
                
            if len(crossroad) > 1:
                for i in crossroad:# 갈림길의 모든 경우의 수 탐색, 앞, 오, 왼 순서로 순회.
                    
                    if i == "forward":
                        to_back_explore(robot)
                        if  update({'new': camera.value}) == "prob_red":
                            camera.stop()
                            camera.unobserve(update_image, names='value')
                            break
                        
                    if i == "right":
                        move(robot, "right")
                        move(robot, "forward")
                        to_back_explore(robot)
                        if  update({'new': camera.value}) == "prob_red":
                            camera.stop()
                            camera.unobserve(update_image, names='value')
                            break
                        move(robot, "back")
                        move(robot, "left")
                    
                    if i == "left":
                        move(robot, "left")
                        move(robot, "forward")
                        to_back_explore(robot)
                        if  update({'new': camera.value}) == "prob_red":
                            camera.stop()
                            camera.unobserve(update_image, names='value')
                            break
                        move(robot, "back")
                        move(robot, "right")
                    #앞,오,왼 순서로 순회필요
                
            
            
            camera.stop()
            camera.unobserve(update_image, names='value')
    total_move(total_check)
            
total_check = list()
trever = list()